In [1]:
import numpy as np


In [2]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        arches[k]=[int(i),int(j)]
        k+=1
  return NA, arches


def obtain_weight(C,arc_idxs,arch):
  k=arc_idxs.index(arch)
  weight=C[k]
  return weight

In [77]:

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])


C = np.array([2, 2, 2, 5, 2, 1, 2])
Aeq, arc_idxs = NN2NA(NN)
beq = np.array([1, 0, 0, 0, 0, -1])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

w = np.ones(NN.shape[1])*np.inf #weights initialization
w[0]=0

precedents=[None]*NN.shape[1]  #precedents initialization

#print('## Optimizer inputs ## \n'
#      'Cost vector: %s \n\n '
#      'Arches in order:\n%s \n\n'
#      'A_eq Node-Arc matrix:\n%s \n\n'
#      'b_eq demand-supply vector: %s \n\n'
#      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

stop=0
arches_to_explore=arc_idxs.copy() #it is neccesary to convert the array to int
next_arches_to_explore=arches_to_explore.copy()
nodes_to_explore=[0]
count=0

print("nodes_to_explore     ",nodes_to_explore)
print("arches_to_explore    ",arches_to_explore)
print("w                    ",w)
print("precedents           ",precedents)

while (stop==0 and len(nodes_to_explore)>0):  #The idea is to go through all the nodes and arches. Stop is a flag to avoid infinite loops.
  next_nodes_to_explore=[]
  for node_i in nodes_to_explore: #The algorithm goes through the actual nodes to explore
    for arch_i in arches_to_explore:  #It will find the next neighbours of the actual node
      node_start=(arch_i[:][0]) #node_start has the number of the node of origin
      if node_start==node_i:
        node_end=(arch_i[:][1]) #node_end has the number of the node of destination
        potential_weight=w[node_start] + obtain_weight(C,arc_idxs,arch_i) #The potential weight is calculated. Then it is compared to the actual weight indexed.
        if w[node_end] > potential_weight:  #Here is the comparisson. if the actual weight is bigger than the potential_weight in this iteration, then...
          precedents[node_end]=node_start  #now the precedent for the next node is the node that is explored in this iteration
          w[node_end]=potential_weight #there is a new weight for this node
        if node_end<(NN.shape[1]-1):next_nodes_to_explore.append(node_end)  #If the next node is not the last, there will be an other iteration
        next_arches_to_explore.remove(arch_i) #Remove the arches that were explored
  nodes_to_explore=next_nodes_to_explore.copy()
  arches_to_explore=next_arches_to_explore.copy()
  print("######################### STEP",count,"###############################")
  print("nodes_to_explore     ",nodes_to_explore)
  print("arches_to_explore    ",arches_to_explore)
  print("w                    ",w)
  print("precedents           ",precedents)

  count+=1
  if (count>100): #break the bucle after 100 iterations
    stop=1 
    print("Algorithm stoped because it counted 100 iterations!!!")


print("###################### RESULT ##################################")
k=NN.shape[1]-1 #The idea is to start in the last node and find the way back to the origin
path=[]
s_out=""
while (k!=None):
  path.insert(0,k)
  k=precedents[k]

for i in path:
  if i==0:
    s_out=s_out+"s->"
  elif i==NN.shape[1]-1:
    s_out=s_out+"t"
  else:
    s_out=s_out+str(i+1)+"->"
print(s_out)

nodes_to_explore      [0]
arches_to_explore     [[0, 1], [0, 2], [1, 3], [1, 5], [2, 4], [3, 5], [4, 5]]
w                     [ 0. inf inf inf inf inf]
precedents            [None, None, None, None, None, None]
######################### STEP 0 ###############################
nodes_to_explore      [1, 2]
arches_to_explore     [[1, 3], [1, 5], [2, 4], [3, 5], [4, 5]]
w                     [ 0.  2.  2. inf inf inf]
precedents            [None, 0, 0, None, None, None]
######################### STEP 1 ###############################
nodes_to_explore      [3, 4]
arches_to_explore     [[3, 5], [4, 5]]
w                     [0. 2. 2. 4. 4. 7.]
precedents            [None, 0, 0, 1, 2, 1]
######################### STEP 2 ###############################
nodes_to_explore      []
arches_to_explore     []
w                     [0. 2. 2. 4. 4. 5.]
precedents            [None, 0, 0, 1, 2, 3]
###################### RESULT ##################################
s->2->4->t
